In [ ]:
#| default_exp core

# Core

Useful functions and utilities for LibreGrid.

In [94]:
#| export
import httpx
import asyncio
import os
import requests
import random
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [46]:
# Fetching environment variables
load_dotenv()
solar_key = os.environ['API_SOLAR_KEY']

In [ ]:
#| export
def get_smart_meter_data():
    "Python function to get smart meter data from Octopus Home Mini API"
    OE_API_KEY = os.getenv("OE_API_KEY")
    # TODO

In [3]:
#| export
def get_solar_building_insights(solar_key, latitude, longitude):
    url = 'https://solar.googleapis.com/v1/buildingInsights:findClosest'
    params = {
    'location.latitude': latitude,
    'location.longitude': longitude,
    'requiredQuality': 'LOW',
    'key':solar_key
    }
    response = requests.get(url, params=params)
    return response.json()

In [4]:
## test = get_solar_building_insights(solar_key=solar_key, latitude=37.4449739, longitude=-122.13914659999998)

In [29]:
def fetch_meter_usage(MPAN_import,serial_number):
    api_key = os.environ['phil_key']
    url = f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN_import}/meters/{serial_number}/consumption/"
    params = {
        "period_from":"2024-01-01 00:00:00",
        "period_to":"2024-07-30 00:00:00",
        "page_size":"20"
    }
    response = requests.get(url,auth=(api_key,''), params = params)
    return response.json()
    

In [30]:
test = fetch_meter_usage(os.environ['MPAN_import'],os.environ['serial_number'])

In [97]:
#| export
async def fetch_latest_meter_data():
    "Function to get last hour of meter consumption data" 
    api_key = os.environ['phil_key']
    mpan = os.environ['MPAN_import']
    serial_num = os.environ['serial_number']

    end_time = datetime.now()
    start_time = end_time - timedelta(hours=10)
    url = f"https://api.octopus.energy/v1/electricity-meter-points/{mpan}/meters/{serial_num}/consumption/"
    params = {
        #"period_from":start_time.isoformat(),
        #"period_to":end_time.isoformat(),
        #'group_by':'hour',
        "page_size":"2"
    }
    async with httpx.AsyncClient() as client:
        response = await client.get(url, auth=(api_key, ''), params=params)
        if response.status_code == 200:
            data = response.json()
            return data['results'] if data['results'] else None
        else:
            print(f"Request failed with status code: {response.status_code}")
            return None
    

In [134]:
#| export
def round_now_to_last_half_hour():
    "Helper function to round current time to last half hour"
    dt = datetime.now()
    rounded = dt.replace(second=0, microsecond=0)
    if rounded.minute >= 30:
        rounded = rounded.replace(minute=30)
    else:
        rounded = rounded.replace(minute=0)
    return rounded

In [141]:
#| export
def dummy_meter(page_size=2):
    "Meter data simulator that just generates random consumption values at every full half hour, at a given interval"
    responses = []
    for item in range(0,page_size): 
        start_time = round_now_to_last_half_hour() - timedelta(minutes=30*(item+1))
        end_time = round_now_to_last_half_hour() - timedelta(minutes=30*(item))
        entry = {
            'consumption': round(random.uniform(0, 0.3), 3),
            'interval_start': start_time.isoformat(),
            'interval_end': end_time.isoformat()
        }
        responses.append(entry)
    return responses
        
    